In [559]:
import time
from sage.rings.polynomial.hilbert import hilbert_poincare_series

# computes degree of regularity for a homogeneous ideal
def hdegree_reg(I):
    G = I.groebner_basis()
    LT = Ideal([g.lt() for g in G])
    HS = hilbert_poincare_series(LT)
    delta = HS.numerator().degree()
    l = HS.denominator().degree()
    return(delta - l + 1)


# computes F^top
def f_top(I, R):
    f_list = I.gens()
    new_ring.<h> = PolynomialRing(R, order = 'degrevlex')
    homogenized = [new_ring(poly.homogenize()) for poly in f_list]
    final = [R(poly.subs(h=0)) for poly in homogenized] 
    return(final)


# computes degree of regularity in the nonhomogeneous case
def degree_reg(I, R):
    J = ideal(f_top(I,R))
    return hdegree_reg(J)

  
def test_rowsp(f, f_list, d, ring):
    mons = get_mons(d, ring)
    M = mac_matrix(f_list, d, ring)
    done = False
    while not done:
        rowsp = M.row_space()
        to_be_added = []
        for row in M.rref().rows():
            g = 0
            for (coeff, mon) in zip(row, mons):
                g += coeff * mon
            max_test_deg = d - g.degree()
            if (g != 0) and (max_test_deg > 0):
                for mon in get_mons(max_test_deg, ring):
                    test_p = mon*g
                    test_vec = vector([test_p.monomial_coefficient(moon) for moon in mons])
                    if test_vec not in rowsp:
                        to_be_added.append(test_vec)
        if to_be_added == []:
            done = True
        else:
            new_rows = M.rows() + to_be_added
            M = Matrix(new_rows)
    f_vec = vector([ring(f).monomial_coefficient(mon) for mon in mons])
    return f_vec in M.row_space()


def get_mons(d, ring, leq = True):
    if leq:
        mons = []
        for i in range(d+1):
            mons += [ring({tuple(a):1}) for a in WeightedIntegerVectors(i,[1 for gen in ring.gens()])]
    else:
        mons = [ring({tuple(a):1}) for a in WeightedIntegerVectors(d,[1 for gen in ring.gens()])]
    mons.sort()
    mons.reverse()
    return mons


def deg_fall(f, f_list, ring):
    test_deg = f.degree()
    while True:
        print(test_deg, time.strftime("%H:%M:%S", time.localtime()))
        if test_rowsp(f, f_list, test_deg, ring):
            return test_deg
        test_deg += 1
        
        
def lfd(f_list, ring):
    fall_degrees = []
    I = ideal(f_list)
    B = I.groebner_basis()
    print(B)
    for f in B:
        print(f)
        d = deg_fall(f, f_list, ring)
        if d > f.degree():
            fall_degrees.append(d)
    if fall_degrees == []:
        return 0
    else:
        return max(fall_degrees)
    
    
# Compute the Macaulay matrix in degree d of a system f = [f1,...,fs]
def mac_matrix(f,d,ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse() # for reasons I don't understand, the .sort() method respects the chosen order but reverses it
    col_labs = []
    for deg in range(d + 1):
        segment = []
        for poly in f:
            for mon in mons:
                label = ring(mon)*ring(poly)
                if label.degree() == deg:
                    segment.append(label)
        col_labs += segment
    return matrix([[label.monomial_coefficient(mon) for mon in mons] for label in col_labs])


# Takes an RREF Macaulay matrix as input and returns a list of the corresponding polynomials
def recover_polys(M, d, ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse()
    B = []
    for row in M.rows():
        g = 0
        for (coeff,mon) in zip(row,mons):
            g += coeff*mon
        if g != 0:
            B.append(g)
    if B == []:
        B = [ring(0)]
    return Sequence(B) # Sage doesn't know how to work with the output list unless it is a "Sequence" of polynomials


# Now instead of using mac_matrix followed by recover_polys, we ask for the new polynomials directly:
def mac_basis(f,d,ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse() # for reasons I don't understand, the .sort() method respects the chosen order but reverses it
    col_labs = []
    for deg in range(d + 1):
        segment = []
        for poly in f:
            for mon in mons:
                label = ring(mon)*ring(poly)
                if label.degree() == deg:
                    segment.append(label)
        col_labs += segment
    M = matrix([[label.monomial_coefficient(mon) for mon in mons] for label in col_labs]).rref()
    B = []
    for row in M.rows():
        g = 0
        for (coeff,mon) in zip(row,mons):
            g += coeff*mon
        if g != 0:
            B.append(g)
    if B == []:
        B = [ring(0)]
    return Sequence(B)


# Now define a function that calculates a Gröbner basis and the solving degree (per Caminata and Gorla, 2021)
def mac_grob(f,ring):
    test_deg = 0
    while True:
        B = mac_basis(f, test_deg, ring)
        print(test_deg, B, B.is_groebner(), ideal(B) == ideal(f))
        if B.is_groebner() and ideal(B) == ideal(f):
            return (test_deg,B)
        test_deg += 1
        

# Modified version to compute the solving degree per Caminata and Gorla, 2023
def sdeg(f, ring):
    d = 0
    while True:
        d_level_base = []
        mons = get_mons(d, ring)
        M = mac_matrix(f, d, ring)
        done = False
        while not done:
            rowsp = M.row_space()
            to_be_added = []
            for row in M.rref().rows():
                g = 0
                for (coeff, mon) in zip(row, mons):
                    g += coeff * mon
                max_test_deg = d - g.degree()
                if (g != 0) and (max_test_deg > 0):
                    for mon in get_mons(max_test_deg, ring):
                        test_p = mon*g
                        test_vec = vector([test_p.monomial_coefficient(moon) for moon in mons])
                        if test_vec not in rowsp:
                            to_be_added.append(test_vec)
            if to_be_added == []:
                done = True
                for row in M.rref().rows():
                    g = 0
                    for (coeff, mon) in zip(row, mons):
                        g += coeff * mon
                    if g != 0:
                        d_level_base.append(g)
                if d_level_base == []:
                    d_level_base = [ring(0)]
                check_1 = Sequence(d_level_base).is_groebner()
                check_2 = (ideal(d_level_base) == ideal(f))
                print(d, Sequence(d_level_base), check_1, check_2)
                if check_1 and check_2:
                    return (d, Sequence(d_level_base))
                d += 1
            else:
                new_rows = M.rows() + to_be_added
                M = Matrix(new_rows)

In [560]:
# Example 4.1
R.<x,y> = PolynomialRing(GF(11), order = "lex")
f1 = x*y + y
f2 = y^2 - 1
f3 = x^10 - 1
I = Ideal(f1,f2,f3)
print(I.groebner_basis(), '\n')
print(mac_grob([f1,f2,f3],R), '\n')
print(sdeg([f1,f2,f3],R), '\n')

[x + 1, y^2 - 1] 

0 [0] True False
1 [0] True False
2 [x*y + y, y^2 - 1] False True
3 [x^2*y - y, x*y^2 + 1, x*y + y, x + 1, y^3 - y, y^2 - 1] True True
(3, [x^2*y - y, x*y^2 + 1, x*y + y, x + 1, y^3 - y, y^2 - 1]) 

0 [0] True False
1 [0] True False
2 [x*y + y, y^2 - 1] False True
3 [x^3 + 1, x^2*y - y, x^2 - 1, x*y^2 + 1, x*y + y, x + 1, y^3 - y, y^2 - 1] True True
(3, [x^3 + 1, x^2*y - y, x^2 - 1, x*y^2 + 1, x*y + y, x + 1, y^3 - y, y^2 - 1]) 



In [7]:
# Continuing Example 4.1, checking regularity of F^h
S.<x,y,h> = PolynomialRing(GF(11), order = "lex")
h_list = [f.homogenize() for f in [f1,f2,f3]]
print(ideal(h_list).groebner_basis())
len(ideal(h_list)._singular_().mres(0).betti())/4

[x^10 - h^10, x*y + y*h, x*h^2 + h^3, y^2 - h^2]


11

In [8]:
# Continuing Example 4.1, checking regularity of in(F^h)
S.<x,y,h> = PolynomialRing(GF(11), order = "lex")
h_list = [f.homogenize() for f in [f1,f2,f3]]
Fh = ideal(h_list)
in_list = [f.lt() for f in Fh.groebner_basis()]
len(ideal(in_list)._singular_().mres(0).betti())/4

11

In [9]:
# Example 4.2
R.<x,y,z> = PolynomialRing(GF(7), order = "lex")
f1 = x*y + y
f2 = y^2 - 1
f3 = z^6 - 1
I = Ideal(f1,f2,f3)
print(I.groebner_basis(), '\n')
print(mac_grob([f1,f2,f3],R), '\n')
print(sdeg([f1,f2,f3],R), '\n')

[x + 1, y^2 - 1, z^6 - 1] 

0 [0] True False
1 [0] True False
2 [x*y + y, y^2 - 1] False False
3 [x^2*y - y, x*y^2 + 1, x*y*z + y*z, x*y + y, x + 1, y^3 - y, y^2*z - z, y^2 - 1] True False
4 [x^3*y + y, x^2*y^2 - 1, x^2*y*z - y*z, x^2*y - y, x^2 - 1, x*y^3 + y, x*y^2*z + z, x*y^2 + 1, x*y*z^2 + y*z^2, x*y*z + y*z, x*y + y, x*z + z, x + 1, y^4 - 1, y^3*z - y*z, y^3 - y, y^2*z^2 - z^2, y^2*z - z, y^2 - 1] True False
5 Polynomial Sequence with 36 Polynomials in 3 Variables True False
6 Polynomial Sequence with 61 Polynomials in 3 Variables True True
(6, Polynomial Sequence with 61 Polynomials in 3 Variables) 

0 [0] True False
1 [0] True False
2 [x*y + y, y^2 - 1] False False
3 [x^3 + 1, x^2*y - y, x^2*z - z, x^2 - 1, x*y^2 + 1, x*y*z + y*z, x*y + y, x*z^2 + z^2, x*z + z, x + 1, y^3 - y, y^2*z - z, y^2 - 1] True False
4 Polynomial Sequence with 26 Polynomials in 3 Variables True False
5 Polynomial Sequence with 45 Polynomials in 3 Variables True False
6 Polynomial Sequence with 72 Polynom

In [10]:
# Continuing Example 4.2, checking regularity of F^h
S.<x,y,z,h> = PolynomialRing(GF(7), order = "lex")
h_list = [f.homogenize() for f in [f1,f2,f3]]
len(ideal(h_list)._singular_().mres(0).betti())/4

8

In [11]:
# Continuing Example 4.2, checking regularity of in(F^h)
S.<x,y,h> = PolynomialRing(GF(7), order = "lex")
h_list = [f.homogenize() for f in [f1,f2,f3]]
Fh = ideal(h_list)
in_list = [f.lt() for f in Fh.groebner_basis()]
len(ideal(in_list)._singular_().mres(0).betti())/4

8

In [12]:
# Example 4.3
R.<x,y> = PolynomialRing(GF(13), order = "lex")
f1 = x^2 - 1
f2 = y^2 - 1
I = Ideal(f1,f2)
print(I.groebner_basis(), '\n')
print(mac_grob([f1,f2],R), '\n')
print(sdeg([f1,f2],R), '\n')

[x^2 - 1, y^2 - 1] 

0 [0] True False
1 [0] True False
2 [x^2 - 1, y^2 - 1] True True
(2, [x^2 - 1, y^2 - 1]) 

0 [0] True False
1 [0] True False
2 [x^2 - 1, y^2 - 1] True True
(2, [x^2 - 1, y^2 - 1]) 



In [13]:
# Continuing Example 4.3, checking regularity of F^h
S.<x,y,h> = PolynomialRing(GF(13), order = "lex")
h_list = [f.homogenize() for f in [f1,f2]]
len(ideal(h_list)._singular_().mres(0).betti())/3

3

In [14]:
# Continuing Example 4.3, checking regularity of in(F^h)
S.<x,y,h> = PolynomialRing(GF(13), order = "lex")
h_list = [f.homogenize() for f in [f1,f2]]
Fh = ideal(h_list)
in_list = [f.lt() for f in Fh.groebner_basis()]
len(ideal(in_list)._singular_().mres(0).betti())/3

3

In [15]:
# Example 4.4
R.<w,x,y,z> = PolynomialRing(GF(5), order = "lex")
f1 = x^2 - 1
f2 = x*y - 1
f3 = w^5 - w
f4 = w^4*z^4 - 1
I = Ideal(f1,f2,f3,f4)
print(I.groebner_basis(), '\n')
print(mac_grob([f1,f2,f3,f4],R), '\n')
print(sdeg([f1,f2,f3,f4],R), '\n')

[w^4 - 1, x - y, y^2 - 1, z^4 - 1] 

0 [0] True False
1 [0] True False
2 [x^2 - 1, x*y - 1] False False
3 [w*x^2 - w, w*x*y - w, x^3 - y, x^2*y - y, x^2*z - z, x^2 - 1, x*y^2 - y, x*y*z - z, x*y - 1, x - y] False False
4 Polynomial Sequence with 29 Polynomials in 4 Variables True False
5 Polynomial Sequence with 66 Polynomials in 4 Variables True False
6 Polynomial Sequence with 130 Polynomials in 4 Variables True False
7 Polynomial Sequence with 230 Polynomials in 4 Variables True False
8 Polynomial Sequence with 376 Polynomials in 4 Variables False True
9 Polynomial Sequence with 580 Polynomials in 4 Variables False True
10 Polynomial Sequence with 853 Polynomials in 4 Variables False True
11 Polynomial Sequence with 1206 Polynomials in 4 Variables False True
12 Polynomial Sequence with 1652 Polynomials in 4 Variables False True
13 Polynomial Sequence with 2204 Polynomials in 4 Variables False True
14 Polynomial Sequence with 2876 Polynomials in 4 Variables False True
15 Polynomial S

In [16]:
# Continuing Example 4.4, checking regularity of F^h
S.<w,x,y,z,h> = PolynomialRing(GF(5), order = "lex")
h_list = [f.homogenize() for f in [f1,f2,f3,f4]]
len(ideal(h_list)._singular_().mres(0).betti())/5

13

In [17]:
# Continuing Example 4.4, checking regularity of in(F^h)
S.<w,x,y,z,h> = PolynomialRing(GF(5), order = "lex")
h_list = [f.homogenize() for f in [f1,f2,f3,f4]]
Fh = ideal(h_list)
in_list = [f.lt() for f in Fh.groebner_basis()]
len(ideal(in_list)._singular_().mres(0).betti())/6

17

In [18]:
# Example 2.4
R.<w,x,y,z> = PolynomialRing(QQ, order = "lex")
f1 = x^2 - x
f2 = x*y - 1
f3 = w^6 - w
f4 = w^5*z^5 - 1
I = Ideal(f1,f2,f3,f4)
print(I.groebner_basis(), '\n') # max.Grob.deg = 5
#print(degree_reg(I,R), '\n') # d_reg = 11
print(mac_grob([f1,f2,f3,f4],R), '\n') # solve.deg > 19, this is as far as I've gotten before the kernel dies
print(sdeg([f1,f2,f3,f4],R), '\n') # sd = 11, computation took several hours on a previous attempt, won't repeat here
#print(lfd([f1,f2,f3,f4],R), '\n') # d_F = 11, computation took several hours on a previous attempt, won't repeat here

[w^5 - 1, x - 1, y - 1, z^5 - 1] 

0 [0] True False
1 [0] True False
2 [x^2 - x, x*y - 1] False False
3 [w*x^2 - w*x, w*x*y - w, x^3 - 1, x^2*y - 1, x^2*z - x*z, x^2 - 1, x*y^2 - y, x*y*z - z, x*y - 1, x - 1] False False
4 Polynomial Sequence with 29 Polynomials in 4 Variables True False
5 Polynomial Sequence with 65 Polynomials in 4 Variables True False
6 Polynomial Sequence with 126 Polynomials in 4 Variables True False
7 Polynomial Sequence with 222 Polynomials in 4 Variables True False
8 Polynomial Sequence with 363 Polynomials in 4 Variables True False
9 Polynomial Sequence with 559 Polynomials in 4 Variables True False
10 Polynomial Sequence with 822 Polynomials in 4 Variables False True
11 Polynomial Sequence with 1166 Polynomials in 4 Variables False True
12 Polynomial Sequence with 1603 Polynomials in 4 Variables False True
13 Polynomial Sequence with 2146 Polynomials in 4 Variables False True


KeyboardInterrupt: 

In [ ]:
# Continuing Example 2.4, checking regularity of F^h
S.<w,x,y,z,h> = PolynomialRing(QQ, order = "lex")
h_list = [f.homogenize() for f in [f1,f2,f3,f4]]
print(ideal(h_list).groebner_basis())
len(ideal(h_list)._singular_().mres(0).betti())/5

In [ ]:
# Continuing Example 2.4, checking regularity of in(F^h)
S.<w,x,y,z,h> = PolynomialRing(QQ, order = "lex")
h_list = [f.homogenize() for f in [f1,f2,f3,f4]]
Fh = ideal(h_list)
in_list = [f.lt() for f in Fh.groebner_basis()]
len(ideal(in_list)._singular_().mres(0).betti())/6 # reg(in(F^h)) = 20

In [55]:
# Example 4.1 -- genericity check
S.<x,y,h> = PolynomialRing(GF(11), order = "degrevlex")
f1 = x*y + y
f2 = y^2 - 1
f3 = x^10 - 1
Fh = ideal([f.homogenize() for f in [f1,f2,f3]])
for prime in Fh.saturation(ideal(x,y,h))[0].associated_primes():
    print(prime, '   ', h in prime)

Ideal (y - h, x + h) of Multivariate Polynomial Ring in x, y, h over Finite Field of size 11     False
Ideal (y + h, x + h) of Multivariate Polynomial Ring in x, y, h over Finite Field of size 11     False


In [57]:
# Example 4.2 -- genericity check
R.<x,y,z,h> = PolynomialRing(GF(7), order = "degrevlex")
f1 = x*y + y
f2 = y^2 - 1
f3 = z^6 - 1
Fh = ideal([f.homogenize() for f in [f1,f2,f3]])
for prime in Fh.saturation(ideal(x,y,z,h))[0].associated_primes():
    print(prime, '   ', h in prime)

Ideal (z + 2*h, y - h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z + h, y - h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z + 3*h, y - h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - 2*h, y - h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - h, y - h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - 3*h, y - h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z + 2*h, y + h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z + h, y + h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z + 3*h, y + h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field 

In [61]:
# Example 4.3 -- genericity check
R.<x,y,h> = PolynomialRing(GF(13), order = "degrevlex")
f1 = x^2 - 1
f2 = y^2 - 1
Fh = ideal([f.homogenize() for f in [f1,f2]])
for prime in Fh.saturation(ideal(x,y,h))[0].associated_primes():
    print(prime, '   ', h in prime)

Ideal (y - h, x - h) of Multivariate Polynomial Ring in x, y, h over Finite Field of size 13     False
Ideal (y + h, x - h) of Multivariate Polynomial Ring in x, y, h over Finite Field of size 13     False
Ideal (y - h, x + h) of Multivariate Polynomial Ring in x, y, h over Finite Field of size 13     False
Ideal (y + h, x + h) of Multivariate Polynomial Ring in x, y, h over Finite Field of size 13     False


In [37]:
# Example 4.4 -- genericity check
R.<w,x,y,z,h> = PolynomialRing(GF(5), order = "degrevlex")
f1 = x^2 - 1
f2 = x*y - 1
f3 = w^5 - w
f4 = w^4*z^4 - 1
Fh = ideal([f.homogenize() for f in [f1,f2,f3,f4]])
for prime in Fh.saturation(ideal(w,x,y,z,h))[0].associated_primes():
    print(prime, '   ', h in prime)

Ideal (z - 2*h, y - h, x - h, w - 2*h) of Multivariate Polynomial Ring in w, x, y, z, h over Finite Field of size 5     False
Ideal (z - 2*h, y + h, x + h, w - 2*h) of Multivariate Polynomial Ring in w, x, y, z, h over Finite Field of size 5     False
Ideal (z + h, y - h, x - h, w - 2*h) of Multivariate Polynomial Ring in w, x, y, z, h over Finite Field of size 5     False
Ideal (z + h, y + h, x + h, w - 2*h) of Multivariate Polynomial Ring in w, x, y, z, h over Finite Field of size 5     False
Ideal (z - h, y - h, x - h, w - 2*h) of Multivariate Polynomial Ring in w, x, y, z, h over Finite Field of size 5     False
Ideal (z - h, y + h, x + h, w - 2*h) of Multivariate Polynomial Ring in w, x, y, z, h over Finite Field of size 5     False
Ideal (z + 2*h, y - h, x - h, w - 2*h) of Multivariate Polynomial Ring in w, x, y, z, h over Finite Field of size 5     False
Ideal (z + 2*h, y + h, x + h, w - 2*h) of Multivariate Polynomial Ring in w, x, y, z, h over Finite Field of size 5     False


In [63]:
# Example 2.4 -- genericity check
R.<w,x,y,z,h> = PolynomialRing(QQ, order = "degrevlex")
f1 = x^2 - x
f2 = x*y - 1
f3 = w^6 - w
f4 = w^5*z^5 - 1
Fh = ideal([f.homogenize() for f in [f1,f2,f3,f4]])
for prime in Fh.saturation(ideal(w,x,y,z,h))[0].associated_primes():
    print(prime, '   ', h in prime)

Ideal (y - h, x - h, w*z + z^2 + w*h + z*h + h^2, w^2 - z*h, z^3 - w*h^2) of Multivariate Polynomial Ring in w, x, y, z, h over Rational Field     False
Ideal (y - h, x - h, z^2 - w*h, w^2 + w*z + w*h + z*h + h^2) of Multivariate Polynomial Ring in w, x, y, z, h over Rational Field     False
Ideal (y - h, x - h, w*z - h^2, w^2 + z^2 + w*h + z*h + h^2, z^3 + z^2*h + w*h^2 + z*h^2 + h^3) of Multivariate Polynomial Ring in w, x, y, z, h over Rational Field     False
Ideal (y - h, x - h, w - z, z^4 + z^3*h + z^2*h^2 + z*h^3 + h^4) of Multivariate Polynomial Ring in w, x, y, z, h over Rational Field     False
Ideal (z - h, y - h, x - h, w^4 + w^3*h + w^2*h^2 + w*h^3 + h^4) of Multivariate Polynomial Ring in w, x, y, z, h over Rational Field     False
Ideal (y - h, x - h, w - h, z^4 + z^3*h + z^2*h^2 + z*h^3 + h^4) of Multivariate Polynomial Ring in w, x, y, z, h over Rational Field     False
Ideal (z - h, y - h, x - h, w - h) of Multivariate Polynomial Ring in w, x, y, z, h over Rational Fi

In [38]:
# Example from Minko's Thesis -- genericity check
R.<x,y,z,h> = PolynomialRing(GF(7), order = 'degrevlex')
f1 = x^5 + y^5 + z^5 - 1
f2 = x^3 + y^3 + z^2 - 1
f3 = y^6 - 1
f4 = z^6 - 1
f_list = [f1*f1*f1, f1*f1*f2, f1*f1*f3, f1*f1*f4, f1*f2*f2, f1*f2*f3, f1*f2*f4, f1*f3*f3, f1*f3*f4, f1*f4*f4, f2*f2*f2, f2*f2*f3, f2*f2*f4, f2*f3*f3, f2*f3*f4, f2*f4*f4, f3*f3*f3, f3*f3*f4, f3*f4*f4, f4*f4*f4, x^7 - x, y^7 - y, z^7 - z]
Fh = ideal([f.homogenize() for f in f_list])
for prime in Fh.saturation(ideal(x,y,z,h))[0].associated_primes():
    print(prime, '   ', h in prime)

Ideal (z + h, y - 3*h, x - 2*h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - h, y - 3*h, x + 3*h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z + 3*h, y + h, x) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - h, y + h, x - h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - h, y - 2*h, x + 2*h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z + h, y - 2*h, x - 3*h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - h, y + 3*h, x - 3*h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - h, y - h, x + h) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7     False
Ideal (z - 3*h, y + 2*h, x) of Multivariate Polynomial Ring in x, y, z, h over Finite 

In [20]:
print([f.homogenize() for f in f_list])

[x^15 + 3*x^10*y^5 + 3*x^5*y^10 + y^15 + 3*x^10*z^5 - x^5*y^5*z^5 + 3*y^10*z^5 + 3*x^5*z^10 + 3*y^5*z^10 + z^15 - 3*x^10*h^5 + x^5*y^5*h^5 - 3*y^10*h^5 + x^5*z^5*h^5 + y^5*z^5*h^5 - 3*z^10*h^5 + 3*x^5*h^10 + 3*y^5*h^10 + 3*z^5*h^10 - h^15, x^13 + x^10*y^3 + 2*x^8*y^5 + 2*x^5*y^8 + x^3*y^10 + y^13 + 2*x^8*z^5 + 2*x^5*y^3*z^5 + 2*x^3*y^5*z^5 + 2*y^8*z^5 + x^3*z^10 + y^3*z^10 + x^10*z^2*h + 2*x^5*y^5*z^2*h + y^10*z^2*h + 2*x^5*z^7*h + 2*y^5*z^7*h + z^12*h - x^10*h^3 - 2*x^5*y^5*h^3 - y^10*h^3 - 2*x^5*z^5*h^3 - 2*y^5*z^5*h^3 - z^10*h^3 - 2*x^8*h^5 - 2*x^5*y^3*h^5 - 2*x^3*y^5*h^5 - 2*y^8*h^5 - 2*x^3*z^5*h^5 - 2*y^3*z^5*h^5 - 2*x^5*z^2*h^6 - 2*y^5*z^2*h^6 - 2*z^7*h^6 + 2*x^5*h^8 + 2*y^5*h^8 + 2*z^5*h^8 + x^3*h^10 + y^3*h^10 + z^2*h^11 - h^13, x^10*y^6 + 2*x^5*y^11 + y^16 + 2*x^5*y^6*z^5 + 2*y^11*z^5 + y^6*z^10 - 2*x^5*y^6*h^5 - 2*y^11*h^5 - 2*y^6*z^5*h^5 - x^10*h^6 - 2*x^5*y^5*h^6 - y^10*h^6 - 2*x^5*z^5*h^6 - 2*y^5*z^5*h^6 - z^10*h^6 + y^6*h^10 + 2*x^5*h^11 + 2*y^5*h^11 + 2*z^5*h^11 - h^16, 

In [24]:
print([f.lt() for f in Fh.groebner_basis()])

[x*y*h^20, x*y^2*h^18, y^2*z*h^18, z^2*h^19, x*y^3*h^16, y^3*z*h^16, z^3*h^17, x^2*h^18, x*y^4*h^14, y^4*z*h^14, y*z^3*h^15, x^2*y*h^16, x*y*z*h^16, y*z^2*h^16, x*y^5*h^12, y^6*h^12, y^2*z^3*h^13, x^2*y^2*h^14, x*y^2*z*h^14, y^2*z^2*h^14, x*z^3*h^14, z^4*h^14, x^2*z*h^15, x*z^2*h^15, x*y^5*z*h^10, y^6*z*h^10, x^5*z^2*h^10, x^4*y*z^2*h^10, x^3*y^2*z^2*h^10, x^2*y^3*z^2*h^10, x*y^4*z^2*h^10, y^5*z^2*h^10, x^4*z^3*h^10, x^3*y*z^3*h^10, x^2*y^2*z^3*h^10, x*y^3*z^3*h^10, y^4*z^3*h^10, x^3*z^4*h^10, x^2*y*z^4*h^10, x*y^2*z^4*h^10, y^3*z^4*h^10, x^2*z^5*h^10, x*y*z^5*h^10, y^2*z^5*h^10, y*z^6*h^10, x^6*h^11, x^5*y*h^11, x^4*y^2*h^11, x^3*y^3*h^11, x^5*z*h^11, x^4*y*z*h^11, x^3*y^2*z*h^11, x^2*y^3*z*h^11, x^4*z^2*h^11, x^3*y*z^2*h^11, x^2*y^2*z^2*h^11, x*y^3*z^2*h^11, y^4*z^2*h^11, x^3*z^3*h^11, x^2*y*z^3*h^11, x*y^2*z^3*h^11, y^3*z^3*h^11, x^2*z^4*h^11, x*y*z^4*h^11, y^2*z^4*h^11, x*z^5*h^11, y*z^5*h^11, z^6*h^11, x^5*h^12, x^4*y*h^12, x^3*y^2*h^12, x^2*y^3*h^12, x^4*z*h^12, x^3*y*z*h^12, x^2

In [28]:
# Example 4.4
R.<w,x,y,z> = PolynomialRing(GF(5), order = "degrevlex")
f1 = x^2 - 1
f2 = x*y - 1
f3 = w^5 - w
f4 = w^4*z^4 - 1
I = Ideal(f1,f2,f3,f4)
S.<w,x,y,z,h> = PolynomialRing(GF(5), order = "degrevlex")
h_list = [f.homogenize() for f in [f1,f2,f3,f4]]
Fh = ideal(h_list)
in_list = [f.lt() for f in Fh.groebner_basis()]

In [29]:
h_list

[x^2 - h^2, x*y - h^2, w^5 - w*h^4, w^4*z^4 - h^8]

In [30]:
in_list

[z^4*h^12, w^4*h^8, w*z^4*h^4, w^4*z^4, w^5, y^2*h^2, x*h^2, x^2, x*y]

In [18]:
# Example from Minko's Thesis -- degree of regularity
R.<x,y,z> = PolynomialRing(GF(7), order = 'degrevlex')
f1 = x^5 + y^5 + z^5 - 1
f2 = x^3 + y^3 + z^2 - 1
f3 = y^6 - 1
f4 = z^6 - 1
f_list = [f1*f1*f1, f1*f1*f2, f1*f1*f3, f1*f1*f4, f1*f2*f2, f1*f2*f3, f1*f2*f4, f1*f3*f3, f1*f3*f4, f1*f4*f4, f2*f2*f2, f2*f2*f3, f2*f2*f4, f2*f3*f3, f2*f3*f4, f2*f4*f4, f3*f3*f3, f3*f3*f4, f3*f4*f4, f4*f4*f4, x^7 - x, y^7 - y, z^7 - z]
degree_reg(ideal(f_list), R)

15

In [28]:
T = ideal(f_top(ideal(f_list),R))

In [30]:
hilbert_poincare_series(T)

3*t^16 + 7*t^15 + 13*t^14 + 19*t^13 + 26*t^12 + 33*t^11 + 36*t^10 + 37*t^9 + 36*t^8 + 33*t^7 + 28*t^6 + 21*t^5 + 15*t^4 + 10*t^3 + 6*t^2 + 3*t + 1

In [32]:
T.hilbert_series()

2*t^14 + 9*t^13 + 19*t^12 + 27*t^11 + 33*t^10 + 36*t^9 + 36*t^8 + 33*t^7 + 28*t^6 + 21*t^5 + 15*t^4 + 10*t^3 + 6*t^2 + 3*t + 1

NameError: name 'hilbert_polynomial' is not defined

In [36]:
f_list

[x^15 + 3*x^10*y^5 + 3*x^5*y^10 + y^15 + 3*x^10*z^5 - x^5*y^5*z^5 + 3*y^10*z^5 + 3*x^5*z^10 + 3*y^5*z^10 + z^15 - 3*x^10 + x^5*y^5 - 3*y^10 + x^5*z^5 + y^5*z^5 - 3*z^10 + 3*x^5 + 3*y^5 + 3*z^5 - 1,
 x^13 + x^10*y^3 + 2*x^8*y^5 + 2*x^5*y^8 + x^3*y^10 + y^13 + 2*x^8*z^5 + 2*x^5*y^3*z^5 + 2*x^3*y^5*z^5 + 2*y^8*z^5 + x^3*z^10 + y^3*z^10 + x^10*z^2 + 2*x^5*y^5*z^2 + y^10*z^2 + 2*x^5*z^7 + 2*y^5*z^7 + z^12 - x^10 - 2*x^5*y^5 - y^10 - 2*x^5*z^5 - 2*y^5*z^5 - z^10 - 2*x^8 - 2*x^5*y^3 - 2*x^3*y^5 - 2*y^8 - 2*x^3*z^5 - 2*y^3*z^5 - 2*x^5*z^2 - 2*y^5*z^2 - 2*z^7 + 2*x^5 + 2*y^5 + 2*z^5 + x^3 + y^3 + z^2 - 1,
 x^10*y^6 + 2*x^5*y^11 + y^16 + 2*x^5*y^6*z^5 + 2*y^11*z^5 + y^6*z^10 - 2*x^5*y^6 - 2*y^11 - 2*y^6*z^5 - x^10 - 2*x^5*y^5 - y^10 - 2*x^5*z^5 - 2*y^5*z^5 - z^10 + y^6 + 2*x^5 + 2*y^5 + 2*z^5 - 1,
 x^10*z^6 + 2*x^5*y^5*z^6 + y^10*z^6 + 2*x^5*z^11 + 2*y^5*z^11 + z^16 - 2*x^5*z^6 - 2*y^5*z^6 - 2*z^11 - x^10 - 2*x^5*y^5 - y^10 - 2*x^5*z^5 - 2*y^5*z^5 - z^10 + z^6 + 2*x^5 + 2*y^5 + 2*z^5 - 1,
 x^11 

In [561]:
def intl(I, ring):
    G = I.groebner_basis()
    in_gens = []
    for gen in G:
        in_gens.append(gen.lt())
    return ideal(in_gens)

In [562]:
def GL_variable_action(g, variable, ring):
    variables = ring.gens()
    n = len(variables)
    row_index = variables.index(variable)
    return sum([variables[i]*g[row_index][i] for i in range(n)])

def GL_function_action(g, f, ring):
    variables = ring.gens()
    sub_dict = {var:GL_variable_action(g, var, ring) for var in variables}
    return f.subs(sub_dict)
    
def GL_action(g, I, ring):
    functions = I.gens()
    return ideal([GL_function_action(g, f, ring) for f in functions])

def random_GL(field, size):
    while True:
        M = random_matrix(QQ, size, size, num_bound = 1000, den_bound = 1).change_ring(field)
        if M.is_invertible():
            break
    return M

def apply_random_GL(I, ring):
    size = len(ring.gens())
    g = random_GL(ring.base_ring(), size)
    return GL_action(g, I, ring)

def attempt_gins(I, ring, n):
    candidates = [intl(apply_random_GL(I, ring), ring) for i in range(n)]
    return candidates

def attempt_gin(I, ring, n):
    candidates = Counter()
    for i in range(n):
        print(i)
        cand = intl(apply_random_GL(I, ring), ring)
        candidates[cand] += 1
        clear_output()
    return max(dict(candidates), key=candidates.get)

def attempt_borel_gins(I, ring, n):
    candidates = []
    for i in range(n):
        cand = intl(apply_random_GL(I, ring), ring)
        if is_fixed(cand, ring):
            candidates.append(cand)
    return candidates

def is_fixed(I, ring, quiet = True):
    n = len(ring.gens())
    var_dict = dict(zip([i for i in range(n)], ring.gens()))
    for mon in I.gens():
        for i in range(n - 1):
            test_var = var_dict[i + 1]
            if test_var.divides(mon):
                new_mon = (mon / test_var) * var_dict[i]
                if new_mon not in I:
                    if not quiet:
                        print(mon, "is in the ideal, but", new_mon, "is not")
                    return False
    return True
    

def make_borel(I, ring):
    new_gens = []
    n = len(ring.gens())
    var_dict = dict(zip([i for i in range(n)], ring.gens()))
    for mon in I.gens():
        d = mon.degree()
        mons = [ring({tuple(a):1}) for a in WeightedIntegerVectors(d,[1 for gen in ring.gens()])]
        new_gens += mons[mons.index(mon):]
    return ideal(ideal(new_gens).groebner_basis())

In [76]:
R.<x,y,z> = PolynomialRing(QQ, order = 'lex')
I = ideal(x^2, x*y, y^3, z*y)
is_fixed(I,R,quiet = False)
attempt_borel_gins(I,R,10)

y*z is in the ideal, but x*z is not


[Ideal (x^2, x*y, x*z, y^3, y^2*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x^2, x*y, x*z, y^3, y^2*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x^2, x*y, x*z, y^3, y^2*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x^2, x*y, x*z, y^3, y^2*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x^2, x*y, x*z, y^3, y^2*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x^2, x*y, x*z, y^3, y^2*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x^2, x*y, x*z, y^3, y^2*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x^2, x*y, x*z, y^3, y^2*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x^2, x*y, x*z, y^3, y^2*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x^2, x*y, x*z, y^3, y^2*z) of Multivariate Polynomial Ring in x, y, z over Rational Field]

In [96]:
R.<x,y,z> = PolynomialRing(QQ)
M = Matrix([[1,2,3],[4,5,6],[7,8,9]])
nums = [x,y,z]
sum([nums[i]*M[0][i] for i in range(3)]) in R

True

In [97]:
GL_variable_action(M, x, R)

x + 2*y + 3*z

In [102]:
f = x^2 + y^2
f.subs(x = GL_variable_action(M,x,R), y = GL_variable_action(M,y,R))

17*x^2 + 44*x*y + 29*y^2 + 54*x*z + 72*y*z + 45*z^2

In [99]:
(x + 2*y + 3*z)^2

x^2 + 4*x*y + 4*y^2 + 6*x*z + 12*y*z + 9*z^2

In [103]:
GL_function_action(M, f, R)

{x: x + 2*y + 3*z, y: 4*x + 5*y + 6*z, z: 7*x + 8*y + 9*z}


17*x^2 + 44*x*y + 29*y^2 + 54*x*z + 72*y*z + 45*z^2

In [91]:
var = x
f.subs({var:y})

y^2

In [106]:
J = ideal(x^2 + y^2)
GL_action(M, J, R)

Ideal (17*x^2 + 44*x*y + 29*y^2 + 54*x*z + 72*y*z + 45*z^2) of Multivariate Polynomial Ring in x, y, z over Rational Field

In [31]:
R.<x,y> = PolynomialRing(QQ, order = "lex")
I = ideal(x^2,y^2)
g = Matrix(QQ, [[1,-1],[1,1]])
intl(GL_action(g,I,R),R)

Ideal (x^2, x*y, y^3) of Multivariate Polynomial Ring in x, y over Rational Field

In [112]:
M = random_matrix(QQ, 2, 2)

In [114]:
M.is_invertible()

True

In [209]:
M = random_GL(GF(7),3)

In [211]:
M.determinant()

5

In [45]:
attempt_gins(I, R, 10)[0]

Ideal (x^2, x*y, y^3) of Multivariate Polynomial Ring in x, y over Rational Field

In [90]:
S.<x,y,z,h> = PolynomialRing(GF(7), order = "degrevlex")
f_list = [x*y*h^20, x*y^2*h^18, y^2*z*h^18, z^2*h^19, x*y^3*h^16, y^3*z*h^16, z^3*h^17, x^2*h^18, x*y^4*h^14, y^4*z*h^14, y*z^3*h^15, x^2*y*h^16, x*y*z*h^16, y*z^2*h^16, x*y^5*h^12, y^6*h^12, y^2*z^3*h^13, x^2*y^2*h^14, x*y^2*z*h^14, y^2*z^2*h^14, x*z^3*h^14, z^4*h^14, x^2*z*h^15, x*z^2*h^15, x*y^5*z*h^10, y^6*z*h^10, x^5*z^2*h^10, x^4*y*z^2*h^10, x^3*y^2*z^2*h^10, x^2*y^3*z^2*h^10, x*y^4*z^2*h^10, y^5*z^2*h^10, x^4*z^3*h^10, x^3*y*z^3*h^10, x^2*y^2*z^3*h^10, x*y^3*z^3*h^10, y^4*z^3*h^10, x^3*z^4*h^10, x^2*y*z^4*h^10, x*y^2*z^4*h^10, y^3*z^4*h^10, x^2*z^5*h^10, x*y*z^5*h^10, y^2*z^5*h^10, y*z^6*h^10, x^6*h^11, x^5*y*h^11, x^4*y^2*h^11, x^3*y^3*h^11, x^5*z*h^11, x^4*y*z*h^11, x^3*y^2*z*h^11, x^2*y^3*z*h^11, x^4*z^2*h^11, x^3*y*z^2*h^11, x^2*y^2*z^2*h^11, x*y^3*z^2*h^11, y^4*z^2*h^11, x^3*z^3*h^11, x^2*y*z^3*h^11, x*y^2*z^3*h^11, y^3*z^3*h^11, x^2*z^4*h^11, x*y*z^4*h^11, y^2*z^4*h^11, x*z^5*h^11, y*z^5*h^11, z^6*h^11, x^5*h^12, x^4*y*h^12, x^3*y^2*h^12, x^2*y^3*h^12, x^4*z*h^12, x^3*y*z*h^12, x^2*y^2*z*h^12, x*y^3*z*h^12, x^3*z^2*h^12, x^2*y*z^2*h^12, x*y^2*z^2*h^12, y^3*z^2*h^12, x^2*z^3*h^12, x*y*z^3*h^12, x*z^4*h^12, y*z^4*h^12, z^5*h^12, x^4*h^13, x^3*y*h^13, x^3*z*h^13, x^2*y*z*h^13, x^2*z^2*h^13, x*y*z^2*h^13, x^3*h^14, x^2*y^3*z^5*h^6, y^5*z^5*h^6, x^2*y^5*z^2*h^7, x^5*y*z^3*h^7, x^5*z^4*h^7, x^3*y^2*z^4*h^7, x^2*y^3*z^4*h^7, x*y^4*z^4*h^7, y^5*z^4*h^7, x^2*y^2*z^5*h^7, x*y^3*z^5*h^7, y^4*z^5*h^7, x^2*y*z^6*h^7, x*y^2*z^6*h^7, x^6*y^2*h^8, x^5*y^3*h^8, x^4*y^4*h^8, x^3*y^5*h^8, x^2*y^6*h^8, x^6*y*z*h^8, x^5*y^2*z*h^8, x^4*y^3*z*h^8, x^3*y^4*z*h^8, x*y^6*z*h^8, x^5*y*z^2*h^8, x^4*y^2*z^2*h^8, x^3*y^3*z^2*h^8, x^2*y^4*z^2*h^8, x*y^5*z^2*h^8, y^6*z^2*h^8, x^5*z^3*h^8, x^4*y*z^3*h^8, x^3*y^2*z^3*h^8, x^2*y^3*z^3*h^8, x*y^4*z^3*h^8, y^5*z^3*h^8, x^4*z^4*h^8, x^3*y*z^4*h^8, x^2*y^2*z^4*h^8, x*y^3*z^4*h^8, y^4*z^4*h^8, x^3*z^5*h^8, x^2*y*z^5*h^8, x*y^2*z^5*h^8, y^3*z^5*h^8, x^2*z^6*h^8, x*y*z^6*h^8, y^2*z^6*h^8, x^6*y*h^9, x^5*y^2*h^9, x^4*y^3*h^9, x^3*y^4*h^9, x*y^6*h^9, x^6*z*h^9, x^5*y*z*h^9, x^4*y^2*z*h^9, x^3*y^3*z*h^9, x^2*y^4*z*h^9, x*z^6*h^9, y^6*z^6*h^3, x^3*y^5*z^3*h^4, x*y^5*z^5*h^4, y^6*z^5*h^4, x^3*y^6*z*h^5, x*y^6*z^3*h^5, x^3*y^3*z^4*h^5, x*y^4*z^5*h^5, x^3*y*z^6*h^5, x^2*y^2*z^6*h^5, x*y^3*z^6*h^5, y^4*z^6*h^5, x^5*y^4*h^6, x^4*y^5*h^6, x^3*y^6*h^6, x^5*y^3*z*h^6, x^3*y^5*z*h^6, x^2*y^6*z*h^6, x^6*y*z^2*h^6, x^5*y^2*z^2*h^6, x^4*y^3*z^2*h^6, x^3*y^4*z^2*h^6, x*y^6*z^2*h^6, x^3*y^3*z^3*h^6, x*y^5*z^3*h^6, y^6*z^3*h^6, x^4*y*z^4*h^6, x^3*y*z^5*h^6, x^3*z^6*h^6, y^3*z^6*h^6, x^6*z^2*h^7, x^3*y^2*z^6*h^3, x*y^4*z^6*h^3, x^6*y*z^3*h^4, x^4*y^3*z^3*h^4, x^4*y*z^5*h^4, x^3*y^2*z^5*h^4, x^6*y^2*z*h^5, x^4*y^4*z*h^5, x^6*z^3*h^5, x^4*y^2*z^3*h^5, x^2*y^4*z^3*h^5, x^4*z^5*h^5, x^3*y^5*z^5, x^4*y^3*z^4*h^2, x^3*y^4*z^4*h^2, x*y^6*z^4*h^2, x^4*y^2*z^5*h^2, x^3*y^3*z^5*h^2, x^2*y^4*z^5*h^2, x^4*z^6*h^3, x^4*y^3*z^5, x^3*y^6*z^2*h, x^6*y^2*z^3*h, x^6*z^4*h^2, x^6*z^5, x^4*y^4*z^2*h, x^4*y^6, x^6*y^3, x^7, y^7, z^7]
Fh = ideal(f_list)
attempt_borel_gins(Fh, S, 100)

[]

In [242]:
for i in range(100):
    for j in range(100):
        if gins[i] == gins[j]:
            print(i,j)

0 0
1 1
1 14
1 51
1 87
2 2
2 21
2 34
2 59
3 3
4 4
5 5
5 98
6 6
6 25
6 26
6 37
6 71
6 74
6 96
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 1
14 14
14 51
14 87
15 15
16 16
17 17
17 35
17 48
17 70
17 89
17 99
18 18
19 19
20 20
21 2
21 21
21 34
21 59
22 22
23 23
24 24
25 6
25 25
25 26
25 37
25 71
25 74
25 96
26 6
26 25
26 26
26 37
26 71
26 74
26 96
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 2
34 21
34 34
34 59
35 17
35 35
35 48
35 70
35 89
35 99
36 36
36 69
37 6
37 25
37 26
37 37
37 71
37 74
37 96
38 38
39 39
40 40
41 41
41 57
42 42
42 80
43 43
44 44
45 45
46 46
47 47
47 86
48 17
48 35
48 48
48 70
48 89
48 99
49 49
50 50
51 1
51 14
51 51
51 87
52 52
53 53
54 54
55 55
56 56
57 41
57 57
58 58
59 2
59 21
59 34
59 59
60 60
61 61
62 62
63 63
64 64
65 65
66 66
67 67
68 68
69 36
69 69
70 17
70 35
70 48
70 70
70 89
70 99
71 6
71 25
71 26
71 37
71 71
71 74
71 96
72 72
73 73
74 6
74 25
74 26
74 37
74 71
74 74
74 96
75 75
75 81
76 76
77 77
78 78
79 79
80 42
80 80
81 75
81 81
82 82
83 83
84 84
85 85
86 47

In [243]:
gins[6]

Ideal (y^2*h^20, x*z^2*h^18, y*z^2*h^18, x*y*h^19, z^6*h^14, x*z^3*h^16, y*z^3*h^16, y^2*z*h^17, x^2*h^18, y^2*z^3*h^14, x*z^4*h^14, y*z^4*h^14, y^2*z^2*h^15, y^3*h^16, x^2*z*h^16, x*y*z*h^16, y^2*z^4*h^12, x*z^5*h^12, y*z^5*h^12, x^2*z^3*h^13, x*y*z^3*h^13, y^4*h^14, x*y^2*z*h^14, y^3*z*h^14, x^2*z^2*h^14, x*y*z^2*h^14, x^2*y*h^15, x*y^2*h^15, x^2*y^4*z*h^10, x*y^5*z*h^10, y^6*z*h^10, x^5*z^2*h^10, x^4*y*z^2*h^10, x^3*y^2*z^2*h^10, x^2*y^3*z^2*h^10, x*y^4*z^2*h^10, y^5*z^2*h^10, x^4*z^3*h^10, x^3*y*z^3*h^10, x^2*y^2*z^3*h^10, x*y^3*z^3*h^10, y^4*z^3*h^10, x^3*z^4*h^10, x^2*y*z^4*h^10, x*y^2*z^4*h^10, y^3*z^4*h^10, x^2*z^5*h^10, x*y*z^5*h^10, y^2*z^5*h^10, x*z^6*h^10, y*z^6*h^10, x^6*h^11, x^5*y*h^11, x^4*y^2*h^11, x^3*y^3*h^11, x^2*y^4*h^11, x*y^5*h^11, y^6*h^11, x^5*z*h^11, x^4*y*z*h^11, x^3*y^2*z*h^11, x^2*y^3*z*h^11, x*y^4*z*h^11, y^5*z*h^11, x^4*z^2*h^11, x^3*y*z^2*h^11, x^2*y^2*z^2*h^11, x*y^3*z^2*h^11, y^4*z^2*h^11, x^3*z^3*h^11, x^2*y*z^3*h^11, x*y^2*z^3*h^11, y^3*z^3*h^11, x^2

In [245]:
intl(Fh,S)

Ideal (x*y*h^20, x*y^2*h^18, y^2*z*h^18, z^2*h^19, x*y^3*h^16, y^3*z*h^16, z^3*h^17, x^2*h^18, x*y^4*h^14, y^4*z*h^14, y*z^3*h^15, x^2*y*h^16, x*y*z*h^16, y*z^2*h^16, x*y^5*h^12, y^6*h^12, y^2*z^3*h^13, x^2*y^2*h^14, x*y^2*z*h^14, y^2*z^2*h^14, x*z^3*h^14, z^4*h^14, x^2*z*h^15, x*z^2*h^15, x*y^5*z*h^10, y^6*z*h^10, x^5*z^2*h^10, x^4*y*z^2*h^10, x^3*y^2*z^2*h^10, x^2*y^3*z^2*h^10, x*y^4*z^2*h^10, y^5*z^2*h^10, x^4*z^3*h^10, x^3*y*z^3*h^10, x^2*y^2*z^3*h^10, x*y^3*z^3*h^10, y^4*z^3*h^10, x^3*z^4*h^10, x^2*y*z^4*h^10, x*y^2*z^4*h^10, y^3*z^4*h^10, x^2*z^5*h^10, x*y*z^5*h^10, y^2*z^5*h^10, y*z^6*h^10, x^6*h^11, x^5*y*h^11, x^4*y^2*h^11, x^3*y^3*h^11, x^5*z*h^11, x^4*y*z*h^11, x^3*y^2*z*h^11, x^2*y^3*z*h^11, x^4*z^2*h^11, x^3*y*z^2*h^11, x^2*y^2*z^2*h^11, x*y^3*z^2*h^11, y^4*z^2*h^11, x^3*z^3*h^11, x^2*y*z^3*h^11, x*y^2*z^3*h^11, y^3*z^3*h^11, x^2*z^4*h^11, x*y*z^4*h^11, y^2*z^4*h^11, x*z^5*h^11, y*z^5*h^11, z^6*h^11, x^5*h^12, x^4*y*h^12, x^3*y^2*h^12, x^2*y^3*h^12, x^4*z*h^12, x^3*y*z*h^1

In [246]:
R.<x,y,z> = PolynomialRing(GF(7), order="degrevlex")
f_list = [x^5 + y^5 + z^5, x^3 + y^3 + z^2, x^7 - x, y^7 - y, z^7 - z]
I = ideal(f_list)

In [12]:
S.<x,y,z,h> = PolynomialRing(GF(7), order="degrevlex")
f_list = [x^5 + y^5 + z^5, x^3 + y^3 + z^2*h, x^7 - x*h^6, y^7 - y*h^6, z^7 - z*h^6]
Fh = ideal(f_list)

In [29]:
is_borel_fixed(intl(Fh,S),S)

False

In [27]:
attempt_gin(Fh,S,500)

Ideal (z^6*h^7, x^2*h^11, x*y*h^11, x*z*h^11, x*y^2*z^2*h^7, y^3*z^2*h^7, x^2*z^3*h^7, x*y*z^3*h^7, y^2*z^3*h^7, x*z^4*h^7, y*z^4*h^7, x*y^2*z*h^8, y^3*z*h^8, x^2*z^2*h^8, x*y*z^2*h^8, y^2*z^2*h^8, x*z^3*h^8, y*z^3*h^8, x^2*y*h^9, x*y^2*h^9, y^3*h^9, x^2*z*h^9, x*y*z*h^9, y^2*z*h^9, x*z^2*h^9, y^2*z^5*h^4, x*z^6*h^4, y*z^6*h^4, x*y^4*z*h^5, y^5*z*h^5, x^2*y^2*z^2*h^5, x*y^3*z^2*h^5, y^4*z^2*h^5, x^2*y*z^3*h^5, x*y^2*z^3*h^5, y^3*z^3*h^5, x^2*z^4*h^5, x*y*z^4*h^5, y^2*z^4*h^5, x*z^5*h^5, x*y^4*h^6, y^5*h^6, x^2*y^2*h^7, x*y^3*h^7, y^4*h^7, x^2*y*z*h^7, x*z^9, y*z^9, x^2*z^6*h^2, x*y*z^6*h^2, y^2*z^6*h^2, x*y^3*z^3*h^3, y^4*z^3*h^3, x^2*y*z^4*h^3, x*y^2*z^4*h^3, y^3*z^4*h^3, x^2*z^5*h^3, x*y*z^5*h^3, x^2*y*z^6, x*y^2*z^6, y^3*z^6, x^2*z^7, x*y*z^7, y^2*z^7, x^2*y*z^5*h, x*y^2*z^5*h, y^3*z^5*h, y^5*z^3, x^2*y^2*z^4, x*y^3*z^4, y^4*z^4, y^5*z^2*h, x^2*y^2*z^3*h, z^7*h, y^7, y^6*z, x*y^4*z^2, y^6*h, x*y^5, x^2*y^3, x^3) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size

In [257]:
max(set(attempts), key=attempts.count)

Ideal (y*z^6*h^6, x*z^2*h^10, x*y*h^11, x*z*h^11, y^2*z^4*h^6, x*z^5*h^6, y^3*z^2*h^7, x^2*z^3*h^7, x*y*z^3*h^7, y^2*z^3*h^7, x*z^4*h^7, y*z^4*h^7, y^3*z*h^8, x^2*z^2*h^8, x*y*z^2*h^8, y^2*z^2*h^8, x*z^3*h^8, y*z^3*h^8, x^2*y*h^9, x*y^2*h^9, y^3*h^9, x^2*z*h^9, x*y*z*h^9, y^2*z*h^9, x^2*h^10, y^2*z^6*h^3, x^2*z^5*h^4, x*y*z^5*h^4, y^2*z^5*h^4, x*z^6*h^4, x^2*y^2*z^2*h^5, x*y^3*z^2*h^5, y^4*z^2*h^5, x^2*y*z^3*h^5, x*y^2*z^3*h^5, y^3*z^3*h^5, x^2*z^4*h^5, x*y*z^4*h^5, x*y^4*h^6, y^5*h^6, x^2*y^2*z*h^6, x^2*y^2*h^7, x*y^3*h^7, y^4*h^7, x^2*y*z*h^7, x*y^2*z*h^7, x*z^9, y*z^9, z^10, x^2*z^6*h^2, x*y*z^6*h^2, x*y^3*z^3*h^3, y^4*z^3*h^3, x^2*y*z^4*h^3, x*y^2*z^4*h^3, y^3*z^4*h^3, x*y^4*z*h^4, y^5*z*h^4, x^2*y*z^6, x*y^2*z^6, y^3*z^6, x^2*z^7, x*y*z^7, y^2*z^7, x^2*y*z^5*h, x*y^2*z^5*h, y^3*z^5*h, y^5*z^3, x^2*y^2*z^4, x*y^3*z^4, y^4*z^4, y^5*z^2*h, x^2*y^2*z^3*h, z^7*h, y^7, y^6*z, x*y^4*z^2, y^6*h, x*y^5, x^2*y^3, x^3) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7

In [259]:
intl(Fh,S)

Ideal (x*y^2*h^10, x*y*h^11, x*z*h^11, y^5*z^2*h^5, y^5*z*h^6, y*z^5*h^6, y^4*z*h^7, y*z^4*h^7, z^5*h^7, x*y^3*h^8, y^3*z*h^8, y*z^3*h^8, z^4*h^8, x^2*y*h^9, x*y*z*h^9, y^2*z*h^9, x*z^2*h^9, z^3*h^9, x^2*h^10, y^5*z^3*h^3, x*y^3*z^3*h^4, x^2*y^2*z^2*h^5, x*y^4*h^6, x*y^3*z*h^6, x*y^2*z^2*h^6, y^3*z^2*h^6, x*y*z^3*h^6, x*z^4*h^6, x*y^2*z*h^7, x*y*z^2*h^7, y^2*z^2*h^7, x*z^3*h^7, y^5*z^4*h, y^4*z^4*h^2, y^2*z^6*h^2, x^2*y^2*z^3*h^3, x*z^6*h^3, x^2*z^4*h^4, y^2*z^4*h^4, z^6*h^4, x^2*y^2*h^6, x^2*z^2*h^6, x^2*z*h^7, y^6*z^2*h, x^2*y^2*z^4*h, x*y^3*z^4*h, x^2*y*z^4*h^2, x*y^2*z^4*h^2, y^3*z^5, x*y^4*z^2*h, y^7, x^2*z^5, x*y*z^5, z^7, x*y^5, x^2*y^3, x^3) of Multivariate Polynomial Ring in x, y, z, h over Finite Field of size 7

In [15]:
attempt_gin(Fh,S,1000)

KeyboardInterrupt: 

In [9]:
print(5)
from IPython.display import clear_output
clear_output()
print(6)

6


In [16]:
from collections import Counter
diction = Counter()

In [18]:
diction['test'] += 1

In [41]:
li = [5,3,7]
f = dict(zip([i for i in range(len(li))],li))
f

{0: 5, 1: 3, 2: 7}

In [372]:
R.<x,y,z> = PolynomialRing(QQ, order = "degrevlex")
I = ideal(y^3, x^2, x*y)
attempt_gins(I, R, 10)

[Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field]

In [375]:
is_generic(I,R)

True

In [373]:
I.hilbert_series()

(t^2 + 2*t + 1)/(-t + 1)

In [55]:
attempt_gins(ideal(y^2*z,x^2,x*z),R,10)

[Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field]

In [58]:
(1 + x + x^2 + x^3)*(x^2 + 2*x + 1)

x^5 + 3*x^4 + 4*x^3 + 4*x^2 + 3*x + 1

In [68]:
is_fixed(ideal(y^2*z,x^2,x*z),R)

y^2*z is in the ideal, but y^3 is not


False

In [104]:
R.<x,y> = PolynomialRing(QQ, order = "degrevlex")
I = ideal(y^3,x^2,x*y)
I.hilbert_series()

t^2 + 2*t + 1

In [107]:
ideal(R(0)).hilbert_series()

1/(t^2 - 2*t + 1)

In [146]:
R.<x,y,z> = PolynomialRing(QQ, order = "degrevlex")
I = ideal(x^2,x*y,y^2*z,z^2)
attempt_borel_gins(I,R,10)

[Ideal (x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Ration

In [140]:
S.<x,y> = PolynomialRing(QQ, order = 'degrevlex')
I = ideal(x^2,y^2)
M = Matrix(QQ, [[2,3],[10,2]])
GL_function_action(M, x*y, S)

20*x^2 + 34*x*y + 6*y^2

In [133]:
(2*x + 3*y)*(10*x + 2*y)

20*x^2 + 34*x*y + 6*y^2

In [134]:
I.groebner_basis()

[x^2, y^2]

In [563]:
# tests conjecture 3 for the degree reverse lex ordering (Sage default)

def is_generic(I,R):
    r = I.dimension()
    if r == 0: 
        return True
    r_gens = list(R.gens())
    M = ideal(r_gens)
    variables_to_check = r_gens[r-2:][::-1]
    variables_checked = []
    for variable in variables_to_check:
        J = I
        for var in variables_checked:
            J = J + ideal(var)
        if J.dimension() == 0:
            return True
        sat = J.saturation(M)[0]
        for prime in sat.associated_primes():
            if variable in prime:
                return False
        variables_checked.append(variable)
    return True

def test_conjecture(f_list, R):
    F = ideal(f_list)
    S = F.homogenize().ring()
    h_list = [S(f.homogenize()) for f in f_list]
    Fh = ideal(h_list)
    reg = Fh._singular_().mres(0).betti().nrows()
    print(reg)
    sd = sdeg(f_list, R)[0]
    return (sd, reg, is_generic(Fh,S))

def test_conjecture_extra(f_list, R):
    F = ideal(f_list)
    S = F.homogenize().ring()
    h_list = [S(f.homogenize()) for f in f_list]
    Fh = ideal(h_list)
    reg = Fh._singular_().mres(0).betti().nrows()
    print(reg, '\n')
    sd = sdeg(f_list, R)[0]
    print('')
    solve_deg = mac_grob(f_list, R)[0]
    return (Fh.dimension(), sd, reg, solve_deg, is_generic(Fh,S))

In [141]:
is_generic(I,S)

True

In [142]:
attempt_gins(I,S,10)

[Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y over Rational Field]

In [144]:
I._singular_().

x^2,
y^2

In [147]:
I = ideal(x*z^2, y*z^2, x^2, x*y, y^2)
is_fixed(I,R)

True

In [148]:
I.hilbert_series()

(2*t^3 - 2*t - 1)/(t - 1)

In [151]:
(1 - x^3)^2*(1 - x^2)^3 / (1 - x)^3

x^9 + 3*x^8 + 3*x^7 - x^6 - 6*x^5 - 6*x^4 - x^3 + 3*x^2 + 3*x + 1

In [158]:
(1 - 2*x^3 + x^6)*(1 + 3*x + 3*x^2 + x^3)

x^9 + 3*x^8 + 3*x^7 - x^6 - 6*x^5 - 6*x^4 - x^3 + 3*x^2 + 3*x + 1

In [152]:
I.dimension()

1

In [156]:
ideal(x^2,x*y,y^2*z,z^2).dimension()

1

In [212]:
J = ideal(y^3,x*y,x^2,x*z)
J.dimension()

1

In [214]:
J.groebner_basis()

[y^3, x^2, x*y, x*z]

In [201]:
(1-x^3)*(1-x^2)^2 / (1-x)^3

x^4 + 3*x^3 + 4*x^2 + 3*x + 1

In [202]:
is_generic(J,R)

True

In [203]:
is_fixed(J,R)

True

In [204]:
attempt_gins(J,R,10)

[Ideal (y^3, x^2, x*y, x*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y, x*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y, x*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y, x*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y, x*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y, x*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y, x*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y, x*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y, x*z) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y, x*z) of Multivariate Polynomial Ring in x, y, z over Rational Field]

In [170]:
apply_random_GL(J,R)

Ideal (-216000*x^3 + 9514800*x^2*y - 139708980*x*y^2 + 683797841*y^3 - 8726400*x^2*z + 256265280*x*y*z - 1881414264*y^2*z - 117515520*x*z^2 + 1725519552*y*z^2 - 527514112*z^3, -43740*x^2 + 592569*x*y + 729468*y^2 - 639552*x*z + 72778*y*z - 680336*z^2, 531441*x^2 + 1207224*x*y + 685584*y^2 + 1227636*x*z + 1394352*y*z + 708964*z^2) of Multivariate Polynomial Ring in x, y, z over Rational Field

In [171]:
M = Matrix([[1,2,3],[4,5,6],[7,8,9]])

In [187]:
GL_action(M,J,R).groebner_basis()

[y^3 + 6*y^2*z + 12*y*z^2 + 8*z^3, x^2 - 4*y^2 - 2*x*z - 16*y*z - 15*z^2, x*y + 2*y^2 + 2*x*z + 7*y*z + 6*z^2]

In [269]:
J

Ideal (x*z^2, y*z^2, z^3, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field

In [211]:
is_fixed(ideal(x^2,x*y,y^2),R)

True

In [257]:
R.<x,y,z> = PolynomialRing(QQ, order = 'degrevlex')
I = ideal(x^4, y^2, z^3, y*z)

In [258]:
I.dimension()

0

In [259]:
is_fixed(I,R,quiet = False)

y^2 is in the ideal, but x*y is not


False

In [260]:
is_generic(I,R)

True

In [261]:
apply_random_GL(I,R).groebner_basis()

[z^6, x*z^4 - 1414064053/1933683780*z^5, y*z^4 - 2147491553/2417104725*z^5, y^2*z^2 + 8954461936573/100912188584970*x*z^3 - 30636771929123/16818698097495*y*z^3 + 51946543662650501119/67304393298631591200*z^4, y^3 - 26639/11910*y^2*z + 709636321/425544300*y*z^2 - 18904001955119/45614093517000*z^3, x^2 - 24025/22201*y^2 - 8327/7146*x*z + 128000395/79324173*y*z - 332619215/1269186768*z^2, x*y - 155/149*y^2 - 26639/35730*x*z + 2062513/2129508*y*z - 612697/4259016*z^2]

In [262]:
J = ideal(x*z^2, y*z^2, z^3, x^2, x*y, y^2)

In [271]:
make_borel(I,R).hilbert_series()

t^2 + 3*t + 1

In [272]:
J.hilbert_series()

3*t^2 + 3*t + 1

In [391]:
R.<x,y,z> = PolynomialRing(QQ, order = 'degrevlex')
I = ideal(x^4, y^3, z^4)
make_borel(I,R).hilbert_series()

6*t^3 + 6*t^2 + 3*t + 1

In [307]:
S.<x,y> = PolynomialRing(QQ, order = 'degrevlex')
I = ideal(x^2,y^3)
make_borel(I,S).hilbert_series()

2*t^2 + 2*t + 1

In [309]:
(1 - x^3)^2*(1 - x^2)/(1 - x)^2

-x^6 - 2*x^5 - 2*x^4 + 2*x^2 + 2*x + 1

In [308]:
make_borel(I,S)

Ideal (x*y^2, y^3, x^2) of Multivariate Polynomial Ring in x, y over Rational Field

In [311]:
ideal(x^3, x*y^2, x^2*y, y^3, x^2).hilbert_series()

2*t^2 + 2*t + 1

In [314]:
(1 - x^3)^4*(1 - x^2)/(1 - x)^2

-x^12 - 2*x^11 - 2*x^10 + 2*x^9 + 6*x^8 + 6*x^7 - 6*x^5 - 6*x^4 - 2*x^3 + 2*x^2 + 2*x + 1

In [322]:
(1 - x^3)^10/(1 - x)^3

-x^27 - 3*x^26 - 6*x^25 + 15*x^23 + 39*x^22 + 27*x^21 - 21*x^20 - 105*x^19 - 105*x^18 - 21*x^17 + 147*x^16 + 189*x^15 + 105*x^14 - 105*x^13 - 189*x^12 - 147*x^11 + 21*x^10 + 105*x^9 + 105*x^8 + 21*x^7 - 27*x^6 - 39*x^5 - 15*x^4 + 6*x^2 + 3*x + 1

In [324]:
make_borel(I,R)

Ideal (x^2*z^2, x*y*z^2, y^2*z^2, x*z^3, y*z^3, z^4, x^3, x^2*y, x*y^2, y^3) of Multivariate Polynomial Ring in x, y, z over Rational Field

In [326]:
(1 - x^4)^15*(1 - x^3)^4/(1 - x)^3

x^69 + 3*x^68 + 6*x^67 + 6*x^66 - 12*x^65 - 48*x^64 - 96*x^63 - 96*x^62 + 57*x^61 + 359*x^60 + 720*x^59 + 720*x^58 - 95*x^57 - 1665*x^56 - 3360*x^55 - 3360*x^54 - 315*x^53 + 5355*x^52 + 10920*x^51 + 10920*x^50 + 2457*x^49 - 12649*x^48 - 26208*x^47 - 26208*x^46 - 8099*x^45 + 22659*x^44 + 48048*x^43 + 48048*x^42 + 17589*x^41 - 31317*x^40 - 68640*x^39 - 68640*x^38 - 27885*x^37 + 33605*x^36 + 77220*x^35 + 77220*x^34 + 33605*x^33 - 27885*x^32 - 68640*x^31 - 68640*x^30 - 31317*x^29 + 17589*x^28 + 48048*x^27 + 48048*x^26 + 22659*x^25 - 8099*x^24 - 26208*x^23 - 26208*x^22 - 12649*x^21 + 2457*x^20 + 10920*x^19 + 10920*x^18 + 5355*x^17 - 315*x^16 - 3360*x^15 - 3360*x^14 - 1665*x^13 - 95*x^12 + 720*x^11 + 720*x^10 + 359*x^9 + 57*x^8 - 96*x^7 - 96*x^6 - 48*x^5 - 12*x^4 + 6*x^3 + 6*x^2 + 3*x + 1

In [327]:
(1 - x^4)^6*(1 - x^3)^4/(1 - x)^3

-x^33 - 3*x^32 - 6*x^31 - 6*x^30 + 3*x^29 + 21*x^28 + 42*x^27 + 42*x^26 + 6*x^25 - 62*x^24 - 126*x^23 - 126*x^22 - 43*x^21 + 99*x^20 + 210*x^19 + 210*x^18 + 90*x^17 - 90*x^16 - 210*x^15 - 210*x^14 - 99*x^13 + 43*x^12 + 126*x^11 + 126*x^10 + 62*x^9 - 6*x^8 - 42*x^7 - 42*x^6 - 21*x^5 - 3*x^4 + 6*x^3 + 6*x^2 + 3*x + 1

In [356]:
make_borel(ideal(z^6,x*y*z),R).hilbert_series()

4*t^5 + 4*t^4 + 4*t^3 + 6*t^2 + 3*t + 1

In [352]:
(1 - x^6)^28/(1 - x)^3

-x^165 - 3*x^164 - 6*x^163 - 10*x^162 - 15*x^161 - 21*x^160 + 48*x^158 + 123*x^157 + 225*x^156 + 354*x^155 + 510*x^154 + 315*x^153 - 231*x^152 - 1128*x^151 - 2376*x^150 - 3975*x^149 - 5925*x^148 - 4950*x^147 - 1050*x^146 + 5775*x^145 + 15525*x^144 + 28200*x^143 + 43800*x^142 + 41850*x^141 + 22350*x^140 - 14700*x^139 - 69300*x^138 - 141450*x^137 - 231150*x^136 - 240120*x^135 - 168360*x^134 - 15870*x^133 + 217350*x^132 + 531300*x^131 + 925980*x^130 + 1024650*x^129 + 827310*x^128 + 333960*x^127 - 455400*x^126 - 1540770*x^125 - 2922150*x^124 - 3415500*x^123 - 3020820*x^122 - 1738110*x^121 + 432630*x^120 + 3491400*x^119 + 7438200*x^118 + 9164925*x^117 + 8671575*x^116 + 5958150*x^115 + 1024650*x^114 - 6128925*x^113 - 15502575*x^112 - 20189400*x^111 - 20189400*x^110 - 15502575*x^109 - 6128925*x^108 + 7931550*x^107 + 26678850*x^106 + 36989865*x^105 + 38864595*x^104 + 32303040*x^103 + 17305200*x^102 - 6128925*x^101 - 37999335*x^100 - 56831850*x^99 - 62626470*x^98 - 55383195*x^97 - 35102025*x^96

In [358]:
make_borel(ideal(z^6, x*y*z),R)

Ideal (y^2*z^4, x*z^5, y*z^5, z^6, x^3, x^2*y, x*y^2, y^3, x^2*z, x*y*z) of Multivariate Polynomial Ring in x, y, z over Rational Field

In [365]:
get_mons(20,R,False).index(x^4*y^13*z^3)+1

74

In [366]:
4 + 13 + (21*20 - 18*17)/2

74

In [370]:
S.<x,y,z,w> = PolynomialRing(QQ, order='degrevlex')
get_mons(20,S,False).index(x*y^3*z^10*w^6)+1

1200

In [377]:
R.<x,y> = PolynomialRing(QQ, order = "degrevlex")
I = ideal((2*x - y)^2, x^2)

In [390]:
I == ideal(x^2, 4*x*y - y^2)

True

In [388]:
ideal(x^2, y^2 - 4*x*y).gens().is_groebner()

False

In [396]:
make_borel(ideal(x^3,y^2,z),R)

Ideal (x, y, z) of Multivariate Polynomial Ring in x, y, z over Rational Field

In [400]:
attempt_gins(ideal(y^3, x^2, z^2), R, 10)

[Ideal (z^5, x*z^3, y*z^3, y^3, y^2*z, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^5, x*z^3, y*z^3, y^3, y^2*z, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^5, x*z^3, y*z^3, y^3, y^2*z, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^5, x*z^3, y*z^3, y^3, y^2*z, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^5, x*z^3, y*z^3, y^3, y^2*z, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^5, x*z^3, y*z^3, y^3, y^2*z, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^5, x*z^3, y*z^3, y^3, y^2*z, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^5, x*z^3, y*z^3, y^3, y^2*z, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^5, x*z^3, y*z^3, y^3, y^2*z, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over

In [401]:
T.<x,y,a,b,c,d,e,f> = PolynomialRing(QQ, order = "degrevlex")
I = ideal(a*x^2 + b*x*y + c*y^2, d*x^2 + e*x*y + f*y^2)

In [402]:
I.groebner_basis()

[y^3*c^2*d - x*y^2*a*c*e - y^3*b*c*e + x*y^2*a*b*f + y^3*b^2*f - y^3*a*c*f, x*y^2*c*d + y^3*c*e - x*y^2*a*f - y^3*b*f, x*y*b*d + y^2*c*d - x*y*a*e - y^2*a*f, x^2*a + x*y*b + y^2*c, x^2*d + x*y*e + y^2*f]

In [403]:
Q.<x,y,a,b,c,d> = PolynomialRing(QQ, order = "degrevlex")
J = ideal(x^2*x^2 + 2*a*b*x*y + b^2*y^2, c^2*x^2 + 2*c*d*x*y + d^2*y^2)
J.groebner_basis()

[y^3*b^3*c^2 - 2*x^3*y^2*b*c*d - 4*y^3*a*b^2*c*d + 2*x^3*y^2*a*d^2 - x^2*y^3*b*d^2 + 4*y^3*a^2*b*d^2, x*y^2*b^2*c^2 + 2*y^3*b^2*c*d - x^3*y^2*d^2 - 2*y^3*a*b*d^2, x*y*a*b*c^2 + 1/2*y^2*b^2*c^2 - x^3*y*c*d - 1/2*x^2*y^2*d^2, x^4 + 2*x*y*a*b + y^2*b^2, x^2*c^2 + 2*x*y*c*d + y^2*d^2]

In [776]:
R.<x,y,z> = PolynomialRing(QQ, order = "degrevlex")
I = ideal(z^4, x*z^2, y*z^2, x^2, x*y, y^2)
attempt_gins(I, R, 10)

[Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of M

In [777]:
J = ideal(x^2, y^2, z^2)
attempt_gins(J, R, 10)

[Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^4, x*z^2, y*z^2, x^2, x*y, y^2) of M

In [778]:
J.hilbert_series()

t^3 + 3*t^2 + 3*t + 1

In [779]:
I.hilbert_series()

t^3 + 3*t^2 + 3*t + 1

In [780]:
I.dimension()

0

In [781]:
J.dimension()

0

In [782]:
(1 - x^4)*(1 - x^3)^2*(1 - x^2)^3 / (1 - x)^3

-x^13 - 3*x^12 - 3*x^11 + x^10 + 7*x^9 + 9*x^8 + 4*x^7 - 4*x^6 - 9*x^5 - 7*x^4 - x^3 + 3*x^2 + 3*x + 1

In [688]:
intl(J,R)

Ideal (x*y*z, x^2, y^2, z^2) of Multivariate Polynomial Ring in x, y, z over Rational Field

In [694]:
is_fixed(J,R,False)

y^2 is in the ideal, but x*y is not


False

In [695]:
y*z in J

False

In [696]:
J1 = ideal(x*y*z)
J1.hilbert_series()

(t^2 + t + 1)/(t^2 - 2*t + 1)

In [697]:
(1-x^3)/(1-x)^3

(-x^2 - x - 1)/(-x^2 + 2*x - 1)

In [701]:
J2 = ideal(x*y*z, x^2)
J2.hilbert_series()

(t^3 - t - 1)/(-t^2 + 2*t - 1)

In [705]:
(1-x^3)*(1-x^2)/(1-x)^3

(x^3 + 2*x^2 + 2*x + 1)/(-x + 1)

In [706]:
J3 = ideal(x*y*z,x^2,y^2)
J3.hilbert_series()

(t^3 - t^2 - 2*t - 1)/(t - 1)

In [707]:
(1-x^3)*(1-x^2)^2/(1-x)^3

x^4 + 3*x^3 + 4*x^2 + 3*x + 1

In [708]:
J.hilbert_series()

3*t^2 + 3*t + 1

In [709]:
(1-x^3)*(1-x^2)^3/(1-x)^3

-x^6 - 3*x^5 - 3*x^4 + 3*x^2 + 3*x + 1

In [712]:
R.<x,y,z> = PolynomialRing(QQ, order = "degrevlex")
I = ideal(y^3, x^2, x*y)
attempt_gins(I, R, 10)

[Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y^3, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field]

In [714]:
J = ideal(x*z^2, x^2, x*y)
attempt_gins(J, R, 10)

[Ideal (x*z^2, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (x*z^2, x^2, x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field]

In [715]:
I.hilbert_series()

(t^2 + 2*t + 1)/(-t + 1)

In [716]:
J.hilbert_series()

(t^4 - t^3 - t^2 + t + 1)/(t^2 - 2*t + 1)

In [718]:
(1-x^3)*(1-x^2)^2/(1-x)^3

x^4 + 3*x^3 + 4*x^2 + 3*x + 1

In [719]:
(1 + x + x^2)*(1 + 2*x + x^2)

x^4 + 3*x^3 + 4*x^2 + 3*x + 1

In [720]:
apply_random_GL(I, R)

Ideal (-267089984*x^3 + 169212288*x^2*y - 35734272*x*y^2 + 2515456*y^3 + 959284368*x^2*z - 405163584*x*y*z + 42781248*y^2*z - 1148460012*x*z^2 + 242531928*y*z^2 + 458314011*z^3, 851929*x^2 + 84916*x*y + 2116*y^2 + 1454648*x*z + 72496*y*z + 620944*z^2, 594412*x^2 - 95904*x*y - 6256*y^2 - 204161*x*z - 142634*y*z - 607548*z^2) of Multivariate Polynomial Ring in x, y, z over Rational Field

In [759]:
M = random_GL(QQ, 3)
N = Matrix(QQ, [[1,0,0],[0,5,0],[2,1,1]])

In [760]:
GL_action(N, I, R)

Ideal (125*y^3, x^2, 5*x*y) of Multivariate Polynomial Ring in x, y, z over Rational Field

In [761]:
I.dimension()

1

In [762]:
J.dimension()

2

In [766]:
(x^2 + 2*x + 1)*(1 + x + x^2 + x^3 + x^4 + x^5 + x^6)

x^8 + 3*x^7 + 4*x^6 + 4*x^5 + 4*x^4 + 4*x^3 + 4*x^2 + 3*x + 1

In [767]:
(x^4 - x^3 - x^2 + x + 1)*(1 + x + x^2 + x^3 + x^4 + x^5 + x^6)^2

x^16 + x^15 + x^12 + 2*x^11 + 3*x^10 + 2*x^9 + 3*x^8 + 6*x^7 + 7*x^6 + 6*x^5 + 5*x^4 + 4*x^3 + 4*x^2 + 3*x + 1

In [770]:
I.hilbert_polynomial()

4

In [785]:
T = ideal(x,y,z^3)
attempt_gins(T, R, 10)

[Ideal (z^3, x, y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^3, x, y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^3, x, y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^3, x, y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^3, x, y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^3, x, y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^3, x, y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^3, x, y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^3, x, y) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (z^3, x, y) of Multivariate Polynomial Ring in x, y, z over Rational Field]

In [786]:
N = ideal(x,y*z, z^3)
attempt_gins(N, R, 10)

[Ideal (y*z^2, y^2, x) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y*z^2, y^2, x) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y*z^2, y^2, x) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y*z^2, y^2, x) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y*z^2, y^2, x) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y*z^2, y^2, x) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y*z^2, y^2, x) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y*z^2, y^2, x) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y*z^2, y^2, x) of Multivariate Polynomial Ring in x, y, z over Rational Field,
 Ideal (y*z^2, y^2, x) of Multivariate Polynomial Ring in x, y, z over Rational Field]

In [788]:
is_generic(N, R)

False

In [791]:
S = N.saturation(ideal(x,y,z))